In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import kornia.augmentation as K
from projects.VolcanoFinder.web.models import MyFirstCNN
import math

In [15]:
cpu_transforms = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = ImageFolder(root='C:/Users/spec/Documents/programming/projects/VolcanoFinder/data/train', transform=cpu_transforms)
val_dataset = ImageFolder(root='C:/Users/spec/Documents/programming/projects/VolcanoFinder/data/val', transform=cpu_transforms)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=12,
    prefetch_factor=6,
    persistent_workers=True,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=16,
    shuffle=False,
    num_workers=12,
    prefetch_factor=6,
    persistent_workers=True,
    pin_memory=True
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

gpu_augmentations = nn.Sequential(
    K.Resize((128, 128)),
    K.RandomHorizontalFlip(p=0.7),
    K.RandomAffine(degrees=0, translate=(0.2, 0.2)),
    K.Normalize(mean=torch.tensor([0.5, 0.5, 0.5], device=device),
                std=torch.tensor([0.5, 0.5, 0.5], device=device))
).to(device)

# ------------------------------- #
# Tried, but disadvantageous: RandRotation, RandGaussianNoise, ColorJitter, RandGauBlur, RandSolarize
# ------------------------------- #

model = MyFirstCNN().to(device)

criterion = nn.BCELoss()

optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

scheduler =  torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.6, patience=3)

# ------------------------------- #

In [16]:
# ------------------------------- #

num_epochs: int = 10
best_val_loss: float = math.inf

for epoch in range(num_epochs):
    # Training loop
    model.train()
    running_loss: float = 0.0
    for images, labels in train_loader:
        images = images.to(device, non_blocking=True)
        labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

        images = gpu_augmentations(images)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {running_loss / len(train_loader):.4f}")

    # Validation Loop
    model.eval()
    val_correct: int = 0
    val_total: int = 0
    val_loss: float = 0.0
    with torch.inference_mode(): # Faster than no_grad by a massive 2 seconds or ~3.076% (1m3s vs 1m5s)
        for images, labels in val_loader:
            images = images.to(device, non_blocking=True)
            labels = labels.float().unsqueeze(1).to(device, non_blocking=True)

            outputs = model(images)
            val_loss += criterion(outputs, labels.float()).item()

            predicted = (outputs > 0.5).float()
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    avg_val_loss: float = val_loss / len(val_loader)
    val_accuracy: float = 100 * val_correct / val_total

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), 'volcano_finder_myownc'
                                       'nn_best.pth')
        print(f" ------------- Best model saved | Epoch: {epoch + 1}, Val loss: {avg_val_loss:.4f} ------------- ")

    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%")

    scheduler.step(avg_val_loss)

# ------------------------------- #

Epoch 1/10, Training Loss: 0.8591
 ------------- Best model saved | Epoch: 1, Val loss: 0.6851 ------------- 
Validation Loss: 0.6851, Accuracy: 62.30%
Epoch 2/10, Training Loss: 0.5289
Validation Loss: 0.7022, Accuracy: 63.93%
Epoch 3/10, Training Loss: 0.4471
 ------------- Best model saved | Epoch: 3, Val loss: 0.5919 ------------- 
Validation Loss: 0.5919, Accuracy: 72.95%
Epoch 4/10, Training Loss: 0.4306
Validation Loss: 0.6202, Accuracy: 65.30%
Epoch 5/10, Training Loss: 0.3911
Validation Loss: 0.6450, Accuracy: 56.56%
Epoch 6/10, Training Loss: 0.4081
 ------------- Best model saved | Epoch: 6, Val loss: 0.5907 ------------- 
Validation Loss: 0.5907, Accuracy: 61.20%
Epoch 7/10, Training Loss: 0.4062
 ------------- Best model saved | Epoch: 7, Val loss: 0.5255 ------------- 
Validation Loss: 0.5255, Accuracy: 80.05%
Epoch 8/10, Training Loss: 0.3494
Validation Loss: 0.5385, Accuracy: 76.78%
Epoch 9/10, Training Loss: 0.3234
Validation Loss: 0.6637, Accuracy: 61.20%
Epoch 10/10,

In [17]:
# ------------------------------- #

test_data_path = 'C:/Users/spec/Documents/programming/projects/VolcanoFinder/data/test_images'

# Create a dataset and DataLoader for the test set
# Use the same transforms as the val set
test_dataset = ImageFolder(root=test_data_path, transform=cpu_transforms)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

loaded_model = MyFirstCNN().to(device)

loaded_model.load_state_dict(torch.load('volcano_finder_myowncnn_best.pth'))

loaded_model.to(device)
loaded_model.eval()

print("Model loaded successfully.")

test_correct: int = 0
test_total: int = 0
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        outputs = loaded_model(images)
        predicted = (outputs > 0.5).float()
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

accuracy: float = 100 * test_correct / test_total
print(f"Test Accuracy: {accuracy:.2f}%")

# ------------------------------- #

Model loaded successfully.
Test Accuracy: 79.78%
